# T2.1 - Solution
## Authors:
- Leonardo Kaplan 1212509
- Nino Fabrizio Tiriticco Lizardo 1113203

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
# Pacotes usados
import pandas as pd
from github import Github
import time
import os
import base64
import difflib


In [ ]:
git = Github("leokaplan",'senha',per_page=100)
#print(git.get_rate_limit())
#print(time.localtime(git.rate_limiting_resettime))
#print(git.per_page)
commits_pag = git.get_user('facebook').get_repo('react').get_commits()

In [ ]:
#print(commits_pag.totalCount)

### Capturando dados, salvando em arquivo e lendo depois

In [ ]:
commits = []
for c in commits_pag[0:1000]:
    commits.insert(len(commits),c)
    #print(c.files)
    if(len(commits)%100==0):
        print(len(commits))

In [ ]:
#commits

In [ ]:
import pickle
def save(commits):
    with open('outfile.json', 'wb') as fp:
        pickle.dump(commits, fp)
def load():
    with open ('outfile.json', 'rb') as fp:
        commits = pickle.load(fp)
        print(len(commits))
    return commits

In [ ]:
#commits = commits[0:10]
#save(commits)
#commits = load()
#commits


In [ ]:
for c in commits[0:100]:
    print(c.files)
    if c.files is None:
        print(commits.index(c))

## Análise prévia dos dados:

#### Meu formato de dados  foi se modificando a medida que via que nao era conveniente. A unica grande mudanca foi transformar tempo de timestamp absoluto para delta relativo. Dessa forma foi possivel prever ocorrencias fora do conjunto de treinamento.

In [ ]:
#df = pd.DataFrame()
d = {'auth':[],'date':[],'adate':[],'numfiles':[],'files':[],'message':[]}
#print(d.loc)
names = []
exts = []
lastdate = None
#lm = None

for commit in reversed(commits[0:100]):
    print(commits.index(commit))
    #d['loc'] += [commit.commit.message.split(' ')[0]]
    #d['typ'] += [commit.commit.message.split(' ')[1]]
    if not commit.commit.author.name in names:
        names.append(commit.commit.author.name)
    d['auth'] += [names.index(commit.commit.author.name)]
    
    if lastdate is None:
        lastdate = commit.commit.author.date.timestamp()
        firstdate = lastdate
    d['date'] += [commit.commit.author.date.timestamp()-lastdate]
    d['adate'] += [commit.commit.author.date]
    lastdate = commit.commit.author.date.timestamp()
    #print(commit.commit.author.date)
    
    extensions = []
    if not commit.files is None:  
        for f in commit.files:
            name,ext = os.path.splitext(f.filename)
            if not ext in exts:
                exts.append(ext)
            extensions += [exts.index(ext)]
        d['files'] += [extensions]
        d['numfiles'] += [len(commit.files)]
    else:
        print(commits.index(commit))
        d['files'] += [""]
        d['numfiles'] += [0]
    #if lm is None:
    #    lm = commit.commit.message
    #d['message'] += [difflib.SequenceMatcher(None, lm, commit.commit.message).ratio()]
    #lm = commit.commit.message
    d['message'] += [commit.commit.message]
    #print(type(str(d['message'][0])),str(d['message'][0]))
    #d.
    #committer = commit.committer
    #files = commit.files
    #df.append(d)
df = pd.DataFrame(d)
df.reset_index(inplace=True)

In [ ]:
for e in exts:
    df[e] = pd.Series([k.count(exts.index(e)) for k in df['files']], index=df.index)
    df[e] = np.multiply(df[e], 1/df['numfiles'])
df

In [ ]:
#dates = matplotlib.dates.date2num(df['date'])
#dates = pd.Timestamp(d['date']).to_pydatetime()
#matplotlib.pyplot.plot_date(d['date'], d['auth'])
#range(0,len(d['date']))
import datetime
fig, ax = plt.subplots(figsize=(20,1))
da = firstdate
dates = []
for ds in d['date']:
    da += ds
    dates.append(datetime.datetime.fromtimestamp(da))

ax.scatter(dates, [1]*len(dates), c=[0]*len(dates),marker='2', s=30)
fig.autofmt_xdate()


### Importei este pedaco de codigo do ultimo trabalho. Fiz leves modificacoes.

In [ ]:
# modelos de classificação
from sklearn import linear_model,datasets,svm,tree,neural_network
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier

#função de utilidade para separar teste e treinamento
from sklearn.model_selection import train_test_split

#funções de utilidade para metricas
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, precision_score, recall_score,f1_score

#funções de utilidade para seleção de features
from sklearn.feature_selection import VarianceThreshold,SelectKBest,chi2
import matplotlib.pyplot as plt
import seaborn as sns
import math

# isto é para usarmos markdown no meio de uma célula Python
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
    
import random

def present(x,y,h):
    fig = plt.figure(figsize = (15,15))
    size = len(data.columns)
    for i in range(0,size-1):
        ax = fig.add_subplot(math.ceil(math.sqrt(size-1)), math.ceil(math.sqrt(size-1)), i+1)
        ax.scatter(x.iloc[:,i], y,  color='red')
        ax.scatter(x.iloc[:,i], h, color='blue')
    plt.show()
def test(regr,X_train,Y_train,X_test,Y_test):
    regr.fit(X_train, Y_train)
    Y_pred = regr.predict(X_test)
    metrics = {
       'mse': mean_squared_error(Y_test, Y_pred),
       'r2':r2_score(Y_test, Y_pred),
       #'accuracy':accuracy_score(Y_test, Y_pred), 
       #'precision':precision_score(Y_test, Y_pred, average='macro'), 
       #'recall':recall_score(Y_test, Y_pred, average='macro'),
       #'f1':f1_score(Y_test, Y_pred, average='macro')
    }
    #metrics={}
    return regr,metrics
methods = [
        linear_model.LinearRegression(),
        svm.SVC(),
        linear_model.SGDClassifier(max_iter=1000,tol=1e-3,loss="huber"),
        #NearestCentroid(),
        KNeighborsClassifier(n_neighbors=5),
        tree.DecisionTreeRegressor(),
        tree.DecisionTreeClassifier(),
        neural_network.MLPClassifier(solver='lbfgs')
    ]
#função que testa todos os métodos, retornando uma lista com as métricas
def test_all(X_train,Y_train,X_test,Y_test):
    metrics = []
    for method in methods:
        #print(method.__class__.__name__)
        #print()
        model,metric = test(method,X_train,Y_train,X_test,Y_test)
        #print()
        metrics = metrics + [metric]
    return metrics

# Perguntas:
## 1) Quando sera o proximo commit? (Ou melhor, quando serao os proximos N?)

In [ ]:
X = df['index'].values.reshape(-1, 1)#[[i,d['date'][i].timestamp()] for i in range(0,len(d['date']))]
Y = df['date']#.values.reshape(len(df),1)
print(np.shape(X),np.shape(Y))
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.25)#nao usar randomstate com valores sequenciais
print(np.shape(X_train),np.shape(Y_train))

In [ ]:
metrics = test_all(X_train,Y_train,X_test,Y_test)
metrics

### Todos os metodos tem muito erro.
#### Vamos ensaiar com modelo linear, arvore de decisao e rede neural.

## vamos tentar prever os indices len(data) ate len(data)+N-1, com N=50

In [ ]:
N=50
#datetime.datetime.fromtimestamp(1.5276342e+09)
model,metric = test(methods[0],X_train,Y_train,X_test,Y_test)
datesk = model.predict(np.arange(len(df),len(df)+N-1).reshape(-1, 1))
#datesk = [datetime.datetime.fromtimestamp(d) for d in datesk]
#datesk

In [ ]:
#df['date'].array()
#df['date'].loc[len(df)] = datesk
vals = np.append(df['date'].values, datesk)
#print(np.shape(vals))
#vals

In [ ]:
fig, ax = plt.subplots(figsize=(20,1))
da = firstdate
dates = []
for d in vals:
    da += d
    dates.append(datetime.datetime.fromtimestamp(da))

ax.scatter(dates, [1]*len(dates), c=['b']*len(df)+['r']*len(datesk),marker='2', s=30)
fig.autofmt_xdate()

## nao ficou bom, depois de um tempo deduzi que eh pq nao treinei nestei valores, entao vou perguntar na faixa treinada 

In [ ]:
model,metric = test(methods[0],X_train,Y_train,X_test,Y_test)
datesk = model.predict(np.arange(0,50).reshape(-1, 1))
vals = np.append(df['date'].values, datesk)
fig, ax = plt.subplots(figsize=(20,1))
da = firstdate
dates = []
for d in vals:
    da += d
    dates.append(datetime.datetime.fromtimestamp(da))

ax.scatter(dates, [1]*len(dates), c=['b']*len(df)+['r']*len(datesk),marker='2', s=30)
fig.autofmt_xdate()

In [ ]:
model,metric = test(methods[-2],X_train,Y_train,X_test,Y_test)
datesk = model.predict(np.arange(0,50).reshape(-1, 1))
vals = np.append(df['date'].values, datesk)
fig, ax = plt.subplots(figsize=(20,1))
da = firstdate
dates = []
for d in vals:
    da += d
    dates.append(datetime.datetime.fromtimestamp(da))

ax.scatter(dates, [1]*len(dates), c=['b']*len(df)+['r']*len(datesk),marker='2', s=30)
fig.autofmt_xdate()
ref_dates = dates

In [ ]:
model,metric = test(methods[-1],X_train,Y_train,X_test,Y_test)
datesk = model.predict(np.arange(0,50).reshape(-1, 1))
vals = np.append(df['date'].values, datesk)
fig, ax = plt.subplots(figsize=(20,1))
da = firstdate
dates = []
for d in vals:
    da += d
    dates.append(datetime.datetime.fromtimestamp(da))

ax.scatter(dates, [1]*len(dates), c=['b']*len(df)+['r']*len(datesk),marker='2', s=30)
fig.autofmt_xdate()

### A rede produz resultados de forma muito inconsistente. As vezes 'e algo distribuido uniformemente, as vezes tudo esta concentrado e as vezes fica bastante 'realista' no sentido de pontos espalhados em intervalos parecidos com a serie ate entao

### A partir de agora meu modelo de escolha eh a arvore de decisao.

#### Para fim de demonstracao, o proximo commit:

In [ ]:
model,metric = test(methods[-2],X_train,Y_train,X_test,Y_test)
datesk = model.predict(np.arange(0,1).reshape(-1, 1))
vals = np.append(df['date'].values, datesk)
da = firstdate
for d in vals:
    da += d
datetime.datetime.fromtimestamp(da)

### plot da arvore com os valores de validacao (teste = verde, previsao = vermelho)

In [ ]:
model,metric = test(methods[-2],X_train,Y_train,X_test,Y_test)
datesk = model.predict(X_test)
vals1 = np.append(df['date'].values, datesk)
vals2 = np.append(df['date'].values, Y_test)
fig, ax = plt.subplots(figsize=(20,1))
da = firstdate
dates1 = []
for d in vals1:
    da += d
    dates1.append(datetime.datetime.fromtimestamp(da))
da = firstdate
dates2 = []
for d in vals2:
    da += d
    dates2.append(datetime.datetime.fromtimestamp(da))
ax.scatter(dates1, [1]*len(df)+[2]*len(datesk), c=['b']*len(df)+['g']*len(datesk),marker='2', s=30)
ax.scatter(dates2, [1]*len(df)+[1]*len(datesk), c=['b']*len(df)+['r']*len(datesk),marker='2', s=30)
fig.autofmt_xdate()
ref_dates = dates

### 2) Vamos tentar prever autor do proximo commit

In [ ]:
X = df['index'].values.reshape(-1, 1)
Y = df['auth']
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=random.randrange(0,100),test_size=0.25)
model,metric = test(methods[-2],X_train,Y_train,X_test,Y_test)
Pnames = model.predict(np.arange(0,50).reshape(-1, 1))
[names[v] for v in Pnames]

In [ ]:
vals = np.append(df['auth'].values, Pnames)
fig, ax = plt.subplots(figsize=(20,1))

ax.scatter(ref_dates, vals, c=['b']*len(df)+['r']*len(datesk),marker='2', s=30)
fig.autofmt_xdate()

In [ ]:
X = df['index'].values.reshape(-1, 1)
Y = df['auth']
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=random.randrange(0,100),test_size=0.25)
model,metric = test(methods[-2],X_train,Y_train,X_test,Y_test)
Pnames = model.predict(np.arange(100,150).reshape(-1, 1))
vals = np.append(df['auth'].values, Pnames)
fig, ax = plt.subplots(figsize=(20,1))
da = firstdate
#dates = []
#for d in dates:
#    da += d
#    dates.append(d...atetime.datetime.fromtimestamp(da))

ax.scatter(dates, vals, c=['b']*len(df)+['r']*len(datesk),marker='2', s=30)
fig.autofmt_xdate()

### Como mostrado acima, quando tratamos de series nao temporais, podemos (devemos?) prever os proximos indices.

### 3) Vamos tentar prever a mensagem do proximo commit

In [ ]:
X = df['index'].values.reshape(-1, 1)
Y = df['message']
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=random.randrange(0,100),test_size=0.25)
model,metric = test(methods[0],X_train,Y_train,X_test,Y_test)
Pmess = model.predict(np.ara nge(100,150).reshape(-1, 1))
vals = np.append(df['message'].values, Pmess)
vals
#fig, ax = plt.subplots(figsize=(20,1))
#da = firstdate
#dates = []
#for d in dates:
#    da += d
#    dates.append(d...atetime.datetime.fromtimestamp(da))

#ax.scatter(dates, vals, c=['b']*len(df)+['r']*len(datesk),marker='2', s=30)
#fig.autofmt_xdate()

In [ ]:
exts
extsto

In [ ]:
X = df['index'].values.reshape(-1, 1)

Y = df[exts]
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=random.randrange(0,100),test_size=0.25)
model,metric = test(methods[0],X_train,Y_train,X_test,Y_test)
Pext = model.predict(np.arange(0,50).reshape(-1, 1))
dfext = pd.DataFrame(Pext,columns=exts)
vals = df[exts].append(dfext)
#vals
fig, ax = plt.subplots(figsize=(20,1))
#da = firstdate
#dates = []
#for d in dates:
#    da += d
#    dates.append(d...atetime.datetime.fromtimestamp(da))
colors = ['k','b','r','m','y','g','c']
for e in exts:
    ax.scatter(ref_dates, vals[e], c=[colors[exts.index(e)]]*len(ref_dates),marker='o', s=30)
#fig.autofmt_xdate()